In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the directories for source images and where to move them
source_dir = '/content/drive/MyDrive/Processed_IMG_NEW'
processed_dir = '/content/drive/MyDrive/Processed_IMG_NEW'


# Create train and test directories
train_dir = '/content/drive/MyDrive/Processed_IMG_NEW/train'
test_dir = '/content/drive/MyDrive/Processed_IMG_NEW/test'

batch size = 16 , epcohs = 25

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 16
EPOCHS = 25
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 859s 17s/step - accuracy: 0.3124 - loss: 8.5995 - val_accuracy: 0.3527 - val_loss: 8.3971 - learning_rate: 2.0000e-05
Epoch 2/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 977ms/step - accuracy: 0.3623 - loss: 8.2933 - val_accuracy: 0.3285 - val_loss: 8.0198 - learning_rate: 3.0000e-05
Epoch 3/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 934ms/step - accuracy: 0.4083 - loss: 7.8796 - val_accuracy: 0.3188 - val_loss: 7.5283 - learning_rate: 4.0000e-05
Epoch 4/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 991ms/step - accuracy: 0.3935 - loss: 7.3752 - val_accuracy: 0.3237 - val_loss: 6.9554 - learning_rate: 5.0000e-05
Epoch 5/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 955ms/step - accuracy: 0.3845 - loss: 6.7876 - val_accuracy: 0.3140 - val_loss: 6.3241 - learning_rate: 6.0000e-05
Epoch 6/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 41s 957ms/step - accuracy: 0.4941 - loss: 6.1538 - val_accuracy: 0.3092 - val_loss: 5.7463 - learning_rate: 6.0000e-05
Epoch 7/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.4580 -


Classification Report:
              precision    recall  f1-score   support

          AD       0.85      0.65      0.74        69
        EMCI       0.70      0.83      0.76        69
        LMCI       0.70      0.74      0.72        69

    accuracy                           0.74       207
   macro avg       0.75      0.74      0.74       207
weighted avg       0.75      0.74      0.74       207


Confusion Matrix:
[[45 10 14]
 [ 4 57  8]
 [ 4 14 51]]
Accuracy for AD: 65.22%
Accuracy for EMCI: 82.61%
Accuracy for LMCI: 73.91%


batch size = 16 , epcohs = 50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 16
EPOCHS = 50
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 494s 7s/step - accuracy: 0.3651 - loss: 8.5931 - val_accuracy: 0.3333 - val_loss: 8.3905 - learning_rate: 2.0000e-05
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 965ms/step - accuracy: 0.3460 - loss: 8.2957 - val_accuracy: 0.3382 - val_loss: 8.0036 - learning_rate: 3.0000e-05
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 945ms/step - accuracy: 0.3904 - loss: 7.8755 - val_accuracy: 0.3527 - val_loss: 7.5160 - learning_rate: 4.0000e-05
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 969ms/step - accuracy: 0.4460 - loss: 7.3570 - val_accuracy: 0.3333 - val_loss: 6.9472 - learning_rate: 5.0000e-05
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 41s 957ms/step - accuracy: 0.4171 - loss: 6.7725 - val_accuracy: 0.3382 - val_loss: 6.3166 - learning_rate: 6.0000e-05
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 960ms/step - accuracy: 0.4299 - loss: 6.1471 - val_accuracy: 0.3527 - val_loss: 5.7370 - learning_rate: 6.0000e-05
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 959ms/step - accuracy: 0.5069


Classification Report:
              precision    recall  f1-score   support

          AD       0.83      0.78      0.81        69
        EMCI       0.84      0.74      0.78        69
        LMCI       0.74      0.87      0.80        69

    accuracy                           0.80       207
   macro avg       0.80      0.80      0.80       207
weighted avg       0.80      0.80      0.80       207


Confusion Matrix:
[[54  7  8]
 [ 5 51 13]
 [ 6  3 60]]
Accuracy for AD: 78.26%
Accuracy for EMCI: 73.91%
Accuracy for LMCI: 86.96%


batch size = 16 , epcohs = 75

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 16
EPOCHS = 75
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/75


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 492s 7s/step - accuracy: 0.3103 - loss: 8.5819 - val_accuracy: 0.3333 - val_loss: 8.3792 - learning_rate: 2.0000e-05
Epoch 2/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 43s 982ms/step - accuracy: 0.3161 - loss: 8.2853 - val_accuracy: 0.3140 - val_loss: 7.9988 - learning_rate: 3.0000e-05
Epoch 3/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 949ms/step - accuracy: 0.4398 - loss: 7.8658 - val_accuracy: 0.3140 - val_loss: 7.5174 - learning_rate: 4.0000e-05
Epoch 4/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.3935 - loss: 7.3664 - val_accuracy: 0.3768 - val_loss: 6.9508 - learning_rate: 5.0000e-05
Epoch 5/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 39s 999ms/step - accuracy: 0.4187 - loss: 6.7800 - val_accuracy: 0.3140 - val_loss: 6.3233 - learning_rate: 6.0000e-05
Epoch 6/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 971ms/step - accuracy: 0.4637 - loss: 6.1618 - val_accuracy: 0.3527 - val_loss: 5.7492 - learning_rate: 6.0000e-05
Epoch 7/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 957ms/step - accuracy: 0.5257 - 


Classification Report:
              precision    recall  f1-score   support

          AD       0.84      0.78      0.81        69
        EMCI       0.81      0.81      0.81        69
        LMCI       0.77      0.83      0.80        69

    accuracy                           0.81       207
   macro avg       0.81      0.81      0.81       207
weighted avg       0.81      0.81      0.81       207


Confusion Matrix:
[[54  4 11]
 [ 7 56  6]
 [ 3  9 57]]
Accuracy for AD: 78.26%
Accuracy for EMCI: 81.16%
Accuracy for LMCI: 82.61%


batch size = 16 , epcohs = 100

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 629s 11s/step - accuracy: 0.3658 - loss: 8.5956 - val_accuracy: 0.3333 - val_loss: 8.3943 - learning_rate: 2.0000e-05
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 85s 976ms/step - accuracy: 0.3752 - loss: 8.2979 - val_accuracy: 0.3043 - val_loss: 8.0198 - learning_rate: 3.0000e-05
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 958ms/step - accuracy: 0.3512 - loss: 7.8932 - val_accuracy: 0.3333 - val_loss: 7.5361 - learning_rate: 4.0000e-05
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 994ms/step - accuracy: 0.4044 - loss: 7.3847 - val_accuracy: 0.3623 - val_loss: 6.9692 - learning_rate: 5.0000e-05
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.4173 - loss: 6.8016 - val_accuracy: 0.3092 - val_loss: 6.3443 - learning_rate: 6.0000e-05
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 35s 968ms/step - accuracy: 0.4548 - loss: 6.1841 - val_accuracy: 0.3237 - val_loss: 5.7716 - learning_rate: 6.0000e-05
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.551


Classification Report:
              precision    recall  f1-score   support

          AD       0.80      0.83      0.81        69
        EMCI       0.81      0.78      0.79        69
        LMCI       0.78      0.78      0.78        69

    accuracy                           0.80       207
   macro avg       0.80      0.80      0.80       207
weighted avg       0.80      0.80      0.80       207


Confusion Matrix:
[[57  5  7]
 [ 7 54  8]
 [ 7  8 54]]
Accuracy for AD: 82.61%
Accuracy for EMCI: 78.26%
Accuracy for LMCI: 78.26%


batch size = 32 , epochs = 25

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32
EPOCHS = 25
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 675s 20s/step - accuracy: 0.3445 - loss: 8.6267 - val_accuracy: 0.3333 - val_loss: 8.5090 - learning_rate: 2.0000e-05
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.3178 - loss: 8.4585 - val_accuracy: 0.3333 - val_loss: 8.2889 - learning_rate: 3.0000e-05
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.3549 - loss: 8.2207 - val_accuracy: 0.2754 - val_loss: 8.0063 - learning_rate: 4.0000e-05
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.3879 - loss: 7.9203 - val_accuracy: 0.2705 - val_loss: 7.6681 - learning_rate: 5.0000e-05
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4317 - loss: 7.5534 - val_accuracy: 0.2802 - val_loss: 7.2783 - learning_rate: 6.0000e-05
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.4744 - loss: 7.1689 - val_accuracy: 0.3333 - val_loss: 6.9045 - learning_rate: 6.0000e-05
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4723 - loss: 6.7950 -


Classification Report:
              precision    recall  f1-score   support

          AD       0.40      0.68      0.51        69
        EMCI       0.77      0.14      0.24        69
        LMCI       0.44      0.49      0.47        69

    accuracy                           0.44       207
   macro avg       0.54      0.44      0.41       207
weighted avg       0.54      0.44      0.41       207


Confusion Matrix:
[[47  1 21]
 [37 10 22]
 [33  2 34]]
Accuracy for AD: 68.12%
Accuracy for EMCI: 14.49%
Accuracy for LMCI: 49.28%


batch size = 32 , epochs = 50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32
EPOCHS = 50
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb7 (Functional)          │ (None, 10, 10, 2560)        │      64,097,687 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2560)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,311,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,410,458 (249.52 MB)

 Trainable params: 65,099,731 (248.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 519s 14s/step - accuracy: 0.3590 - loss: 8.6126 - val_accuracy: 0.3333 - val_loss: 8.5113 - learning_rate: 2.0000e-05
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.3380 - loss: 8.4545 - val_accuracy: 0.3333 - val_loss: 8.2948 - learning_rate: 3.0000e-05
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.3412 - loss: 8.2152 - val_accuracy: 0.3333 - val_loss: 8.0174 - learning_rate: 4.0000e-05
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.3899 - loss: 7.9303 - val_accuracy: 0.3333 - val_loss: 7.6870 - learning_rate: 5.0000e-05
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.3617 - loss: 7.5778 - val_accuracy: 0.3333 - val_loss: 7.2977 - learning_rate: 6.0000e-05
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.4284 - loss: 7.1805 - val_accuracy: 0.3333 - val_loss: 6.9270 - learning_rate: 6.0000e-05
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4566 - loss: 6.8174 -

batch size = 32 , epochs = 75

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32
EPOCHS = 75
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")


batch size = 32 , epochs = 100

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32
EPOCHS = 100
PATIENCE = 15

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4]
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load EfficientNetB7 model
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Unfreeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = True

# Custom layers on top of EfficientNetB7
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust the number of classes accordingly
])

# Focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    cce = tf.keras.losses.CategoricalCrossentropy()
    def loss(y_true, y_pred):
        cce_loss = cce(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_weight_factor = tf.reduce_sum(alpha * y_true, axis=-1)
        return modulating_factor * alpha_weight_factor * cce_loss
    return loss

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Adjusted learning rate for fine-tuning
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr + 1e-5
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=4, factor=0.3, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("efficientnetb7_ad_model.h5")
